## Label making for Fer2013

#### The main script for making relaxed labels for FER2013 dataset
#### Saving the relaxed labels into .npy file for later model training 


In [1]:
"""
Required packages: numpy and pandas; matplotlib 

"""
import numpy as np
import pandas as pd


In [2]:
"""
Defined functions for label relaxation

1. gumbel function to random sample gumbel variables 

2. softmax function to compute softmax (e.g., logits from [0, 1])

3. gumbelmax function: do not use this function 

4. gumbelsoftmax function: the main function to turn one-hot labels to real-numbered forms
        Input: one-hot labels, e.g., y_train or one-hot(y_train)
                Number of samples: N
        Output: N samples of relaxed labels
"""



def gumbel(*args):
    
    u = np.random.rand(*args)
    return -np.log(-np.log(u))


def softmax(x, axis=None):
    
    x = np.exp(x)
    
    return x/np.sum(x, axis=axis)

def gumbelmax(logits, N=100000):
    
    d = len(logits)
    
    return np.argmax(np.log(logits.reshape(d,1)) + gumbel(d*N).reshape(d,N),axis=0)

def gumbelsoftmax(logits, tau, N=100000):
    
    #d_1 = len(logits[0])
    d_0 = len(logits)
    return softmax((logits.reshape(d_0,1) + 
                    gumbel(d_0*N).reshape(d_0,N))/tau,axis=0)


In [3]:
import data_utils

Using TensorFlow backend.


In [4]:
## Loading Fer2013 dataset from preprocessing data script, i.e., data_utils.py

X_train, y_train, X_test, y_test = data_utils.load_data()

In [5]:
from keras.utils import to_categorical

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test  = np.asarray(X_test)
y_test  = np.asarray(y_test)

## one-hot encoded all the labels for training and testing in y
import keras.backend as K

y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

In [7]:
print y_train.shape

(32181, 7)


In [10]:
%time
tem = [] #creating a holder
for i in range(0,32181): # iterate through all labels(32181 labels in y_train and y_test)
    tem_arr = gumbelsoftmax(y_train[i]*6, tau = 3.5).mean(axis = 1) #take N samples, then average over N 
    #tem_arr_new = np.expand_dims(tem_arr, axis = 0)
    tem = np.concatenate([tem, tem_arr], axis = 0)
    

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 16 µs


In [9]:
#np.save('gc_y_train(tau = 3.5)',tem ) ## save this relaxed labels into .npy format file 